In [1]:
%cd ..

a:\Phong\Crawl_buffett


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time, os
from src.investing import Investing
import pandas as pd
import numpy as np
reverse = False

In [ ]:
df_all = pd.read_csv('A:\Phong\Crawl_buffett\docs\List_company_23052023 - Listing.csv')

path_save = 'data'
miss_count = 0

def check_miss(id_company, path_save, miss_count):
    if not os.path.exists(f"{path_save}/{id_company}/docs/check.csv"):
        return miss_count
    df = pd.read_csv( f"{path_save}/{id_company}/docs/check.csv")  # read file
    for quarter in ['Q1', 'Q2', 'Q3', 'Q4']:
        miss_count += (df[f'Link_{quarter}'] != "[]").sum() - (df[f'download_{quarter}'] == "OK").sum()
    if miss_count > 0:
        print(id_company)
    return miss_count
for id_company in df_all['Symbol']:
    miss_count = check_miss(id_company, path_save, miss_count)
print(miss_count)

In [4]:
def check_miss_file(symbol):
    df = pd.read_csv(fr'Data\{symbol}\docs\check.csv')
    QUARTER = ['Q1', 'Q2', 'Q3', 'Q4']
    df_temp = df[['download_Q1', 'download_Q2', 'download_Q3', 'download_Q4']]
    rows, cols = np.where(df_temp.isna())
    lst_time, lst_quarter = [],  []
    for row, col in zip(rows, cols):
        if df[f'Link_{QUARTER[col]}'][row] != '[]':
            lst_time.append(df['Year'][row])
            lst_quarter.append(QUARTER[col])

    df_new = pd.DataFrame({
                'Symbol': symbol,
                    'Year':lst_time,
                'Quarter': lst_quarter,
                })
    return df_new

df_all = pd.read_csv('A:\Phong\Crawl_buffett\docs\List_company_23052023 - Listing.csv')
df_miss = pd.DataFrame(columns=['Symbol', 'Year', 'Quarter'])
for symbol in df_all['Symbol']:
    if os.path.exists(fr'Data\{symbol}\docs\check.csv'):
        df_temp = check_miss_file(symbol)
        df_miss = pd.concat([df_miss, df_temp])
df_miss['check'] = np.nan
df_miss.to_csv(f'docs\miss_{reverse}.csv', index=False)

In [7]:
def get_link(symbol, year, quarter):
    df_check = pd.read_csv(fr'Data\{symbol}\docs\check.csv')
    links = eval(df_check[f'Link_{quarter}'][df_check['Year'] == year].iloc[0])
    times = eval(df_check[f'Time_{quarter}'][df_check['Year'] == year].iloc[0])
    print('hi')
    for i, time in enumerate(times):
        if "(訂正)" not in time:
            return links[i], time
        
def thread_file():
    df_miss = pd.read_csv(f'docs\miss_{reverse}.csv')
    id = df_miss[df_miss['check'].isna()].index[0]
    symbol = df_miss['Symbol'][id]
    year = df_miss['Year'][id]
    quarter = df_miss['Quarter'][id]
    df_miss['check'][id] = 'Doing'
    df_miss.to_csv(f'docs\miss_{reverse}.csv', index=False)
    link, time = get_link(symbol, year, quarter)
    return link, time

thread_file()

hi


('https://www.buffett-code.com/company/6474/library/a9f8234fe56a641b347e3f/preview',
 '決算短信 (2014/7/3)')

In [3]:
pd.read_csv(fr'Data\6474\docs\check.csv')

,Year,Time_Q1,Link_Q1,Link_pdf_Q1,Time_Q2,Link_Q2,Link_pdf_Q2,Time_Q3,Link_Q3,Link_pdf_Q3,Time_Q4,Link_Q4,Link_pdf_Q4,download_Q1,download_Q2,download_Q3,download_Q4
0,2022,['決算短信 (2023/4/5)'],['https://www.buffett-code.com/company/6474/li...,NaN,[],[],NaN,[],[],NaN,[],[],NaN,OK,NaN,NaN,NaN
1,2021,['決算短信 (2022/4/4)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2022/7/14)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2022/10/6)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2023/1/12)'],['https://www.buffett-code.com/company/6474/li...,NaN,OK,OK,OK,OK
2,2020,['決算短信 (2021/4/2)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2021/7/14)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2021/10/4)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2022/1/13)'],['https://www.buffett-code.com/company/6474/li...,NaN,OK,OK,OK,OK
3,2019,['決算短信 (2020/4/2)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2020/7/15)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2020/10/15)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2021/1/14)'],['https://www.buffett-code.com/company/6474/li...,NaN,OK,OK,OK,OK
4,2018,['決算短信 (2019/4/12)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2019/7/4)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2019/10/3)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2020/1/15)'],['https://www.buffett-code.com/company/6474/li...,NaN,OK,OK,OK,OK
5,2017,['決算短信 (2018/4/5)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2018/7/5)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2018/10/4)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2019/1/15)'],['https://www.buffett-code.com/company/6474/li...,NaN,OK,OK,OK,OK
6,2016,['決算短信 (2017/4/4)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2017/7/5)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2017/10/5)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2018/1/11)'],['https://www.buffett-code.com/company/6474/li...,NaN,OK,OK,OK,OK
7,2015,['決算短信 (2016/4/4)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2016/7/6)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2016/10/3)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2017/1/11)'],['https://www.buffett-code.com/company/6474/li...,NaN,OK,OK,OK,OK
8,2014,['決算短信 (2015/4/2)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2015/7/7)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2015/10/1)'],['https://www.buffett-code.com/company/6474/li...,NaN,['(訂正)決算短信 (2016/1/14)'],['https://www.buffett-code.com/company/6474/li...,NaN,OK,OK,OK,NaN
9,2013,['決算短信 (2014/4/3)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2014/7/3)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2014/10/2)'],['https://www.buffett-code.com/company/6474/li...,NaN,['決算短信 (2015/1/15)'],['https://www.buffett-code.com/company/6474/li...,NaN,OK,NaN,NaN,NaN
